In [1]:
import sys
import os
import platform
import importlib
import glob

import numpy as np
import pandas as pd
import netCDF4
import datetime

if platform.system() == 'Darwin':  # macOS
    base_FP = '/Users/hyunglokkim/Insync/hkim@geol.sc.edu/Google_Drive'
    sys.path.append(base_FP + '/python_modules')
    nc_save_dir = '/Users/hyunglokkim/cpuserver_data'
else:
    base_FP = '/home/h/Insync/hkim@geol.sc.edu/Google Drive'
    sys.path.append(base_FP + '/python_modules')
    nc_save_dir = '/data'
    
import hydroAI.SMOS_IC as hSMOS_IC
import hydroAI.ASCAT_TUW as hASCAT_TUW
import hydroAI.Plot as Plot
import hydroAI.Data as Data
importlib.reload(hSMOS_IC);
importlib.reload(hASCAT_TUW);
importlib.reload(Data);
import warnings

# Ignore runtime warnings
warnings.filterwarnings("ignore")

In [7]:
def find_closest_index(array, value):
    """
    Find the index of the closest value in an array to a given value.
    """
    return np.abs(array - value).argmin()

def crop_3d_data(data_3d, longitude, latitude, bounds):
    """
    Crop 3D data based on specified longitude and latitude min/max values and return cropped data along with
    corresponding longitude and latitude arrays.

    :param data_3d: 3D numpy array to be cropped
    :param longitude: 2D numpy array representing longitude values of data
    :param latitude: 2D numpy array representing latitude values of data
    :param bounds: List of [lon_min, lon_max, lat_min, lat_max]
    :return: Tuple of cropped 3D numpy array, cropped longitude array, cropped latitude array
    """
    lon_min, lon_max, lat_min, lat_max = bounds
    
    # Find the indices for slicing
    lon_min_idx = find_closest_index(longitude[0, :], lon_min)  # Assumes longitude varies along the 1st axis
    lon_max_idx = find_closest_index(longitude[0, :], lon_max)  # Adjust if longitude varies along a different axis
    lat_min_idx = find_closest_index(latitude[:, 0], lat_min)  # Assumes latitude varies along the 0th axis
    lat_max_idx = find_closest_index(latitude[:, 0], lat_max)  # Adjust if latitude varies along a different axis

    # Ensure the min index is less than the max index
    lon_min_idx, lon_max_idx = min(lon_min_idx, lon_max_idx), max(lon_min_idx, lon_max_idx)
    lat_min_idx, lat_max_idx = min(lat_min_idx, lat_max_idx), max(lat_min_idx, lat_max_idx)

    # Slice the 3D data array
    cropped_data = data_3d[lat_min_idx:lat_max_idx+1, lon_min_idx:lon_max_idx+1, :]
    
    # Slice the longitude and latitude arrays
    cropped_longitude = longitude[lat_min_idx:lat_max_idx+1, lon_min_idx:lon_max_idx+1]
    cropped_latitude = latitude[lat_min_idx:lat_max_idx+1, lon_min_idx:lon_max_idx+1]
    
    return cropped_data, cropped_longitude, cropped_latitude

In [6]:
# Define your paths, products, and years
resampled_nc = os.path.join(nc_save_dir, 'extracted_nc')
path_list = ['am', 'pm']
product_list = ['SMOS_IC', 'ASCAT_TUW', 'SMAP_SM']  # Include SMAP_SM in the product list
years = range(2015, 2022)

# Initialize arrays to None
SMOS_SM_3d  = None
SMOS_RFI_3d = None
ASCAT_SM_3d = None
SMAP_SM_3d  = None 
SMAP_tb4_3d = None
# Loop over each year, path, and product
for year in years:
    for path in path_list:
        for product in product_list:
            print(product)
            # Check the product and construct the file path accordingly
            if product in ['SMOS_IC', 'ASCAT_TUW']:
                # File path for SMOS_IC and ASCAT_TUW
                nc_file = os.path.join(resampled_nc, f'{product}_{year}_{path}_R_SPL3SMP.008.nc')
            elif product == 'SMAP_SM':
                # File path for SMAP_SM
                nc_file = os.path.join(resampled_nc, f'SPL3SMP.008_{year}_{path}.nc')
            
            # Open the netCDF file
            nc_data = netCDF4.Dataset(nc_file)
            
            if product == 'SMOS_IC':
                # Extract the SMOS data
                t_SMOS_SM = nc_data.variables['SMOS_SM'][:].data[:,:,1:]
                t_SMOS_RFI = nc_data.variables['SMOS_RFI'][:].data[:,:,1:]
                
                # Concatenate the data
                SMOS_SM_3d = t_SMOS_SM if SMOS_SM_3d is None else np.concatenate((SMOS_SM_3d, t_SMOS_SM), axis=2)
                SMOS_RFI_3d = t_SMOS_RFI if SMOS_RFI_3d is None else np.concatenate((SMOS_RFI_3d, t_SMOS_RFI), axis=2)
            elif product == 'ASCAT_TUW':
                # Extract the ASCAT data
                t_ASCAT_SM = nc_data.variables['ASCAT_SM'][:].data[:,:,1:]
                
                # Concatenate the data
                ASCAT_SM_3d = t_ASCAT_SM if ASCAT_SM_3d is None else np.concatenate((ASCAT_SM_3d, t_ASCAT_SM), axis=2)
            elif product == 'SMAP_SM':
                # Extract the SMAP data
                if path == 'pm':
                    t_SMAP_SM = nc_data.variables['soil_moisture_'+path][:].data[:,:,1:]
                    t_SMAP_tb4 = nc_data.variables['tb_qual_flag_4_'+path][:].data[:,:,1:]
                else:
                    t_SMAP_SM = nc_data.variables['soil_moisture'][:].data[:,:,1:]
                    t_SMAP_tb4 = nc_data.variables['tb_qual_flag_4'][:].data[:,:,1:]
                
                # Concatenate the data
                SMAP_SM_3d = t_SMAP_SM if SMAP_SM_3d is None else np.concatenate((SMAP_SM_3d, t_SMAP_SM), axis=2)
                SMAP_tb4_3d = t_SMAP_tb4 if SMAP_tb4_3d is None else np.concatenate((SMAP_tb4_3d, t_SMAP_tb4), axis=2)
            
            # Close the netCDF file
            nc_data.close()

SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM
SMOS_IC
ASCAT_TUW
SMAP_SM


OSError: [Errno -101] NetCDF: HDF error: '/Users/hyunglokkim/cpuserver_data/extracted_nc/SPL3SMP.008_2021_pm.nc'

In [ ]:
nc_data = netCDF4.Dataset(nc_file)
latitude = nc_data.variables['latitude'][:].data
longitude = nc_data.variables['longitude'][:].data
nc_data.close()
ROK_bound = [125.7, 129.7, 33.9, 38.8]

In [ ]:
ROK_SMAP_SM_3d, ROK_longitude, ROK_latitude = crop_3d_data(SMAP_SM_3d, longitude, latitude, ROK_bound)
ROK_SMAP_tb4_3d, ROK_longitude, ROK_latitude = crop_3d_data(SMAP_tb4_3d, longitude, latitude, ROK_bound)
ROK_SMOS_SM_3d, ROK_longitude, ROK_latitude = crop_3d_data(SMOS_SM_3d, longitude, latitude, ROK_bound)
ROK_SMOS_RFI_3d, ROK_longitude, ROK_latitude = crop_3d_data(SMOS_RFI_3d, longitude, latitude, ROK_bound)
ROK_ASCAT_SM_3d, ROK_longitude, ROK_latitude = crop_3d_data(ASCAT_SM_3d, longitude, latitude, ROK_bound)

In [ ]:
Plot.plot_regional_map(ROK_longitude, ROK_latitude, np.nansum(t,2), '1', 0, 0.5, 0.1, cmap='jet', dem_path=None)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent(ROK_bound, crs=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, color='grey')
mesh = plt.pcolormesh(ROK_longitude, ROK_latitude, np.nanmean(ROK_SMOS_RFI_3d,2), shading='auto', transform=ccrs.PlateCarree())

